In [6]:
import pandas as pd
import requests
import time
from tqdm import tqdm
from collections import Counter


In [2]:
GITHUB_TOKEN = "INSERT_TOKEN"
headers = {
    "Authorization": f"token {GITHUB_TOKEN}",
    "Accept": "application/vnd.github.v3+json"
}

In [7]:
path_to_file = r'output_files\\fix_prs_with_issues_and_files_and_tests.parquet'
prs = pd.read_parquet(path_to_file)

In [8]:
prs

,id,number,user,user_id,agent,title,body,state,created_at,closed_at,...,issues,has_issues,modified_files,modified_files_list,#_of_files,added_tests,modified_tests,removed_tests,renamed_tests,has_modified_test
0,2438086945,88748,iamrajjoshi,33237075,Human,:bug: fix: update how we fetch workflow_id and...,i realized i made a mistake for how i fetch th...,closed,2025-04-03T21:36:59Z,2025-04-04T15:10:57Z,...,[],False,"[{""filename"": ""src/sentry/integrations/opsgeni...","[{'additions': 3, 'changes': 5, 'deletions': 2...",4,0,2,0,0,True
1,2265431531,83085,ArthurKnaus,7033940,Human,fix(org-stats): Require project membership,### Problem\r\n\r\nIf the user is not member o...,closed,2025-01-08T07:47:13Z,2025-01-08T08:49:40Z,...,[82998],True,"[{""filename"": ""static/app/views/organizationSt...","[{'additions': 29, 'changes': 29, 'deletions':...",2,0,1,0,0,True
2,2622011651,94465,bukzor,640328,Human,fix(dev): mktemp: too few X's in template,"For maximum compatibility, busybox mktemp requ...",closed,2025-06-26T18:54:10Z,2025-06-26T19:57:23Z,...,[],False,"[{""filename"": "".envrc"", ""status"": ""modified"", ...","[{'additions': 1, 'changes': 2, 'deletions': 1...",1,0,0,0,0,False
3,2565399631,92785,dashed,139499,Human,fix(billing): Update calculateCategoryPrepaidU...,Closes https://linear.app/getsentry/issue/BIL-...,closed,2025-06-03T22:22:51Z,2025-06-05T18:13:54Z,...,[],False,"[{""filename"": ""static/gsApp/views/subscription...","[{'additions': 6, 'changes': 12, 'deletions': ...",3,0,2,0,0,True
4,2374801945,86438,brendanhsentry,171613822,Human,fix: copy updates to checkout page,closes https://github.com/getsentry/getsentry/...,closed,2025-03-05T22:39:12Z,2025-03-06T16:57:20Z,...,[],False,"[{""filename"": ""static/gsApp/views/amCheckout/s...","[{'additions': 32, 'changes': 49, 'deletions':...",2,0,1,0,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9879,3260452571,1542,devin-ai-integration[bot],158243242,Devin,Fix CSS color assertions in test_lambdas.py,# Fix CSS color assertions in test_lambdas.py\...,closed,2025-07-24T16:14:38Z,2025-07-25T11:29:31Z,...,[],False,"[{""filename"": ""tests/test_lambdas.py"", ""status...","[{'additions': 2, 'changes': 4, 'deletions': 2...",1,0,1,0,0,True
9880,2857103111,2151,devin-ai-integration[bot],158243242,Devin,fix: Initialize storage in StringKnowledgeSource,Fixes #2150\n\n## Issue\nStringKnowledgeSource...,closed,2025-02-17T08:16:22Z,2025-02-25T16:39:20Z,...,[2150],True,"[{""filename"": ""src/crewai/knowledge/source/str...","[{'additions': 51, 'changes': 60, 'deletions':...",2,0,1,0,0,True
9881,2857279950,8459,devin-ai-integration[bot],158243242,Devin,Add missing OpenSSL TLSEXT status response codes,Fixes the build failure in the OpenSSL coexist...,closed,2025-02-17T09:32:13Z,2025-02-17T09:34:11Z,...,[],False,"[{""filename"": ""wolfssl/openssl/ssl.h"", ""status...","[{'additions': 5, 'changes': 5, 'deletions': 0...",1,0,0,0,0,False
9882,2857942945,2,devin-ai-integration[bot],158243242,Devin,fix: improve dark mode input focus and toggle ...,# UI Improvements: Dark Mode Input Focus and T...,closed,2025-02-17T13:57:22Z,2025-02-17T14:40:12Z,...,[],False,"[{""filename"": ""app.js"", ""status"": ""modified"", ...","[{'additions': 10, 'changes': 17, 'deletions':...",2,0,0,0,0,False


In [4]:
def count_reviews_per_user(reviews_list):
    # Se a lista for vazia ou nula, retorna dicionário vazio
    if not reviews_list:
        return {}
    
    # 1. Extrai apenas os logins dos usuários da lista de reviews
    # reviews_list é algo como: [{'user': 'devA', ...}, {'user': 'devA', ...}, {'user': 'devB', ...}]
    users = [r['user'] for r in reviews_list]
    
    # 2. Conta a ocorrência de cada usuário
    # O resultado será algo como: {'devA': 2, 'devB': 1}
    return dict(Counter(users))
def get_pr_reviewers(url):
    try:
        response = requests.get(url, headers=headers)
        
        if response.status_code == 403:
            print("Rate limit atingido. Pausando 60s...")
            time.sleep(60)
            response = requests.get(url, headers=headers)

        if response.status_code == 200:
            reviews_data = response.json()
            
            reviewers_list = []
            for review in reviews_data:
                # Verifica se o objeto 'user' existe (as vezes bots ou usuários deletados retornam null)
                if review.get('user'):
                    reviewers_list.append({
                        'user': review['user']['login'],
                        'state': review['state'],
                        'body': review['body'],
                        'submitted_at': review['submitted_at'] # Útil se quiser saber quando revisou
                    })
            return reviewers_list
            
        else:
            # Se der 404 ou outro erro, retorna lista vazia
            return []
            
    except Exception as e:
        print(f"Erro na URL {url}: {e}")
        return []

In [5]:
closed_prs = prs[prs['state'] != 'open']
closed_prs['number'] = closed_prs['number'].astype(str)

# Cria a URL específica para buscar as reviews daquela PR
closed_prs['reviews_url'] = (
    closed_prs['repo_url'] 
    + '/pulls/' 
    + closed_prs['number'] 
    + '/reviews'
)

C:\Users\Renato\AppData\Local\Temp\ipykernel_9460\3544442098.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  closed_prs['number'] = closed_prs['number'].astype(str)
C:\Users\Renato\AppData\Local\Temp\ipykernel_9460\3544442098.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  closed_prs['reviews_url'] = (


In [6]:
# Aplica a função na coluna que contém os dados brutos das reviews
closed_prs['reviews_data'] = closed_prs['reviews_url'].apply(get_pr_reviewers)
closed_prs['review_counts_map'] = closed_prs['reviews_data'].apply(count_reviews_per_user)

# --- Visualização dos Resultados ---

# Exibe as primeiras linhas para conferir
closed_prs[['number', 'review_counts_map']].head()
# Exemplo de como acessar um dado específico:
# Se quiser saber quantas vezes o usuário 'x' revisou na primeira linha do DF:
# print(closed_prs.iloc[0]['review_counts_map'].get('nome_do_usuario', 0))

C:\Users\Renato\AppData\Local\Temp\ipykernel_9460\1246393160.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  closed_prs['reviews_data'] = closed_prs['reviews_url'].apply(get_pr_reviewers)
C:\Users\Renato\AppData\Local\Temp\ipykernel_9460\1246393160.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  closed_prs['review_counts_map'] = closed_prs['reviews_data'].apply(count_reviews_per_user)


,number,review_counts_map
0,88748,{'GabeVillalobos': 2}
1,83085,{'obostjancic': 1}
2,94465,{'joshuarli': 1}
3,92785,{'isabellaenriquez': 1}
4,86438,"{'isabellaenriquez': 1, 'dashed': 1}"


In [10]:
closed_prs

,id,number,user,user_id,agent,title,body,state,created_at,closed_at,...,modified_files_list,#_of_files,added_tests,modified_tests,removed_tests,renamed_tests,has_modified_test,reviews_url,reviews_data,review_counts_map
0,2438086945,88748,iamrajjoshi,33237075,Human,:bug: fix: update how we fetch workflow_id and...,i realized i made a mistake for how i fetch th...,closed,2025-04-03T21:36:59Z,2025-04-04T15:10:57Z,...,"[{'additions': 3, 'changes': 5, 'deletions': 2...",4,0,2,0,0,True,https://api.github.com/repos/getsentry/sentry/...,"[{'user': 'GabeVillalobos', 'state': 'COMMENTE...",{'GabeVillalobos': 2}
1,2265431531,83085,ArthurKnaus,7033940,Human,fix(org-stats): Require project membership,### Problem\r\n\r\nIf the user is not member o...,closed,2025-01-08T07:47:13Z,2025-01-08T08:49:40Z,...,"[{'additions': 29, 'changes': 29, 'deletions':...",2,0,1,0,0,True,https://api.github.com/repos/getsentry/sentry/...,"[{'user': 'obostjancic', 'state': 'APPROVED', ...",{'obostjancic': 1}
2,2622011651,94465,bukzor,640328,Human,fix(dev): mktemp: too few X's in template,"For maximum compatibility, busybox mktemp requ...",closed,2025-06-26T18:54:10Z,2025-06-26T19:57:23Z,...,"[{'additions': 1, 'changes': 2, 'deletions': 1...",1,0,0,0,0,False,https://api.github.com/repos/getsentry/sentry/...,"[{'user': 'joshuarli', 'state': 'APPROVED', 'b...",{'joshuarli': 1}
3,2565399631,92785,dashed,139499,Human,fix(billing): Update calculateCategoryPrepaidU...,Closes https://linear.app/getsentry/issue/BIL-...,closed,2025-06-03T22:22:51Z,2025-06-05T18:13:54Z,...,"[{'additions': 6, 'changes': 12, 'deletions': ...",3,0,2,0,0,True,https://api.github.com/repos/getsentry/sentry/...,"[{'user': 'isabellaenriquez', 'state': 'APPROV...",{'isabellaenriquez': 1}
4,2374801945,86438,brendanhsentry,171613822,Human,fix: copy updates to checkout page,closes https://github.com/getsentry/getsentry/...,closed,2025-03-05T22:39:12Z,2025-03-06T16:57:20Z,...,"[{'additions': 32, 'changes': 49, 'deletions':...",2,0,1,0,0,True,https://api.github.com/repos/getsentry/sentry/...,"[{'user': 'isabellaenriquez', 'state': 'APPROV...","{'isabellaenriquez': 1, 'dashed': 1}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9879,3260452571,1542,devin-ai-integration[bot],158243242,Devin,Fix CSS color assertions in test_lambdas.py,# Fix CSS color assertions in test_lambdas.py\...,closed,2025-07-24T16:14:38Z,2025-07-25T11:29:31Z,...,"[{'additions': 2, 'changes': 4, 'deletions': 2...",1,0,1,0,0,True,https://api.github.com/repos/reflex-dev/reflex...,"[{'user': 'greptile-apps[bot]', 'state': 'COMM...","{'greptile-apps[bot]': 1, 'Lendemor': 1}"
9880,2857103111,2151,devin-ai-integration[bot],158243242,Devin,fix: Initialize storage in StringKnowledgeSource,Fixes #2150\n\n## Issue\nStringKnowledgeSource...,closed,2025-02-17T08:16:22Z,2025-02-25T16:39:20Z,...,"[{'additions': 51, 'changes': 60, 'deletions':...",2,0,1,0,0,True,https://api.github.com/repos/crewAIInc/crewAI/...,[],{}
9881,2857279950,8459,devin-ai-integration[bot],158243242,Devin,Add missing OpenSSL TLSEXT status response codes,Fixes the build failure in the OpenSSL coexist...,closed,2025-02-17T09:32:13Z,2025-02-17T09:34:11Z,...,"[{'additions': 5, 'changes': 5, 'deletions': 0...",1,0,0,0,0,False,https://api.github.com/repos/wolfSSL/wolfssl/p...,[],{}
9882,2857942945,2,devin-ai-integration[bot],158243242,Devin,fix: improve dark mode input focus and toggle ...,# UI Improvements: Dark Mode Input Focus and T...,closed,2025-02-17T13:57:22Z,2025-02-17T14:40:12Z,...,"[{'additions': 10, 'changes': 17, 'deletions':...",2,0,0,0,0,False,https://api.github.com/repos/jina-ai/deepsearc...,[],{}


In [12]:
closed_prs[['id', 'number', 'user', 'user_id', 'agent', 
       'repo_url', 'html_url',
       'reviews_url', 'reviews_data',
       'review_counts_map']].to_csv('output_files/prs_reviews.csv',index= False)
closed_prs[['id', 'number', 'user', 'user_id', 'agent', 
       'repo_url', 'html_url',
       'reviews_url', 'reviews_data',
       'review_counts_map']].to_parquet('output_files/prs_reviews.parquet',index= False)


In [6]:
path_to_file = r'output_files\prs_reviews.csv'
closed_prs = pd.read_csv(path_to_file)

In [7]:
closed_prs

,id,number,user,user_id,agent,repo_url,html_url,reviews_url,reviews_data,review_counts_map
0,2438086945,88748,iamrajjoshi,33237075,Human,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/88748,https://api.github.com/repos/getsentry/sentry/...,"[{'user': 'GabeVillalobos', 'state': 'COMMENTE...",{'GabeVillalobos': 2}
1,2265431531,83085,ArthurKnaus,7033940,Human,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/83085,https://api.github.com/repos/getsentry/sentry/...,"[{'user': 'obostjancic', 'state': 'APPROVED', ...",{'obostjancic': 1}
2,2622011651,94465,bukzor,640328,Human,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/94465,https://api.github.com/repos/getsentry/sentry/...,"[{'user': 'joshuarli', 'state': 'APPROVED', 's...",{'joshuarli': 1}
3,2565399631,92785,dashed,139499,Human,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/92785,https://api.github.com/repos/getsentry/sentry/...,"[{'user': 'isabellaenriquez', 'state': 'APPROV...",{'isabellaenriquez': 1}
4,2374801945,86438,brendanhsentry,171613822,Human,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/86438,https://api.github.com/repos/getsentry/sentry/...,"[{'user': 'isabellaenriquez', 'state': 'APPROV...","{'isabellaenriquez': 1, 'dashed': 1}"
...,...,...,...,...,...,...,...,...,...,...
9047,3260452571,1542,devin-ai-integration[bot],158243242,Devin,https://api.github.com/repos/reflex-dev/reflex...,https://github.com/reflex-dev/reflex-web/pull/...,https://api.github.com/repos/reflex-dev/reflex...,"[{'user': 'greptile-apps[bot]', 'state': 'COMM...","{'greptile-apps[bot]': 1, 'Lendemor': 1}"
9048,2857103111,2151,devin-ai-integration[bot],158243242,Devin,https://api.github.com/repos/crewAIInc/crewAI,https://github.com/crewAIInc/crewAI/pull/2151,https://api.github.com/repos/crewAIInc/crewAI/...,[],{}
9049,2857279950,8459,devin-ai-integration[bot],158243242,Devin,https://api.github.com/repos/wolfSSL/wolfssl,https://github.com/wolfSSL/wolfssl/pull/8459,https://api.github.com/repos/wolfSSL/wolfssl/p...,[],{}
9050,2857942945,2,devin-ai-integration[bot],158243242,Devin,https://api.github.com/repos/jina-ai/deepsearc...,https://github.com/jina-ai/deepsearch-ui/pull/2,https://api.github.com/repos/jina-ai/deepsearc...,[],{}
